In [1]:
import sys

import torchvision.transforms as T
from IPython.display import Audio
import librosa
import src.custom_transforms as CT
from scipy import signal
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian
from scipy.signal import stft, istft
from src.Dataset import AudioMNIST

In [2]:
sample_rate=22050
n_fft=512
root = '../Data'

transforms = [
    CT.TrimSilence(5),
    CT.TimeStretchFixLength(sample_rate)
]

dataset = AudioMNIST(root, target_sample_rate=sample_rate/2, transform=T.Compose(transforms))

In [3]:
x, _ = dataset[11]
Audio(data=x, rate=sample_rate)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
x.shape

In [ ]:
(3584 - 256)/128

In [16]:
import scipy
import numpy as np
#(x.shape- nperseg) % (nperseg-noverlap) == 0)

f, t, Zxx = scipy.signal.stft(x, sample_rate/2, nperseg = n_fft/2, noverlap = n_fft/4, window='hann')
        # get amplitude
Zxx = Zxx[0:128, :-1]
#Zxx = np.abs(Zxx[0:128, :-1])
#Zxx = np.atleast_3d(Zxx).transpose(2,0,1)
        # convert to decibel
#Zxx = librosa.amplitude_to_db(Zxx, ref = np.max)

In [ ]:
Zxx.shape

In [18]:
def complex_to_2d(tensor: np.ndarray) -> np.ndarray:
    """Converts complex IQ to two channels representing real and imaginary

    Args:
        tensor (:class:`numpy.ndarray`):
            (batch_size, vector_length, ...)-sized tensor.

    Returns:
        transformed (:class:`numpy.ndarray`):
            Expanded vectors
    """

    new_tensor = np.zeros((2, tensor.shape[0]), dtype=np.float64)
    new_tensor[0] = np.real(tensor).astype(np.float64)
    new_tensor[1] = np.imag(tensor).astype(np.float64)
    return new_tensor


In [19]:
datapoint = complex_to_2d(Zxx.flatten())

In [21]:
datapoint=datapoint.reshape(2,128,32)

In [ ]:
datapoint.shape

In [ ]:
Zxxx = np.abs(Zxx)
Zxxx = np.atleast_3d(Zxxx).transpose(2,0,1)
        # convert to decibel
Zxxx = librosa.amplitude_to_db(Zxxx, ref = np.max)
import matplotlib.pyplot as plt
librosa.display.specshow(Zxxx[0])
plt.title("Real")

In [24]:
def twod_to_complex(tensor: np.ndarray):
    """Converts complex IQ to two channels representing real and imaginary

    Args:
        tensor (:class:`numpy.ndarray`):
            (batch_size, vector_length, ...)-sized tensor.

    Returns:
        transformed (:class:`numpy.ndarray`):
            Expanded vectors
    """
    tmp_tensor = tensor.reshape(2, tensor.shape[1]*tensor.shape[2])
    new_tensor = np.zeros((1, tmp_tensor.shape[1]), dtype=np.complex64)
    new_tensor[0] = tmp_tensor[0,:]+ 1j * tmp_tensor[1,:]
    new_tensor=new_tensor.reshape(1, tensor.shape[1],tensor.shape[2])
    return new_tensor


In [ ]:
Zxx_rec=twod_to_complex(datapoint)
Zxx_rec.shape

In [ ]:

_,x = scipy.signal.istft(Zxx_rec, sample_rate/2)
        # get amplitude
x.shape

In [ ]:
Audio(data=x, rate=sample_rate/2)